In [1]:
pip install leafmap

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# -----------------------------
# Utility Function to Read API Responses
# -----------------------------
def read_response(response):
    try:
        return response.json()
    except requests.exceptions.JSONDecodeError:
        print("Invalid JSON response!")
        print("Response content:", response.text)
        return None

# -----------------------------
# Get List of Available Provinces
# -----------------------------
def get_provinces_list():
    response = requests.get("http://127.0.0.1:5000/api/provinces")
    return read_response(response) or []

# -----------------------------
# Get List of Available Municipalities
# -----------------------------
def get_municipalities_list():
    response = requests.get("http://127.0.0.1:5000/api/municipalities")
    return read_response(response) or []

# -----------------------------
# Get List of Available Pollutants
# -----------------------------
def get_pollutants_list():
    response = requests.get("http://127.0.0.1:5000/api/pollutants")
    return read_response(response) or []

# -----------------------------
# Call API to Get Data
# -----------------------------
def get_data(tipo, nome, pollutant):
    url = "http://127.0.0.1:5000/api/DV_7comune" if tipo == "Comune" else "http://127.0.0.1:5000/api/DV_7provincia"
    payload = {
        "var_comune": nome if tipo == "Comune" else None,
        "var_provincia": nome if tipo == "Provincia" else None,
        "var_pollutant": pollutant
    }
    payload = {k: v for k, v in payload.items() if v is not None}
    response = requests.post(url, json=payload)
    data = read_response(response)
    return pd.DataFrame(data) if data else pd.DataFrame()

# -----------------------------
# Dropdowns e Output UI
# -----------------------------
tipo_dropdown = widgets.Dropdown(
    options=["Comune", "Provincia"],
    description="Tipo:",
    value="Comune"
)

nome_dropdown = widgets.Dropdown(
    options=[],
    description="Nome:"
)

pollutant_dropdown = widgets.Dropdown(
    options=get_pollutants_list(),
    description="Inquinante:"
)

output = widgets.Output()

## DV-7 Pollutant comparison over same area
The interface allow the user to select one province or one municipality and visualize the time series of each pollutant, the time series will be obtained calculating for each pollutant the daily average over the selected area.

In [ ]:
# -----------------------------
# Aggiorna lista nomi (comuni o province)
# -----------------------------
def aggiorna_lista_nomi(change):
    if change['new'] == "Comune":
        nome_dropdown.options = get_municipalities_list()
    else:
        nome_dropdown.options = get_provinces_list()

tipo_dropdown.observe(aggiorna_lista_nomi, names='value')

# Primo caricamento
aggiorna_lista_nomi({'new': tipo_dropdown.value})

# -----------------------------
# Mostra dati e grafico
# -----------------------------
def mostra_dati(b):
    with output:
        clear_output()
        tipo = tipo_dropdown.value
        nome = nome_dropdown.value
        inquinante = pollutant_dropdown.value

        if not nome:
            print("Seleziona un nome valido.")
            return
        if not inquinante:
            print("Seleziona un inquinante.")
            return

        df = get_data(tipo, nome, inquinante)
        if df.empty:
            print(f"Nessun dato trovato per {tipo} '{nome}' e inquinante '{inquinante}'.")
            return

        display(df)

        # Plot
        df['data'] = pd.to_datetime(df['data'])
        df = df.sort_values('data')
        plt.figure(figsize=(10, 4))
        plt.plot(df['data'], df['valore'], marker='o')
        plt.title(f"Serie temporale di {inquinante} - {tipo}: {nome}")
        plt.xlabel("Data")
        plt.ylabel("Valore medio")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

# -----------------------------
# Bottone e Display
# -----------------------------
button = widgets.Button(description="Mostra dati")
button.on_click(mostra_dati)

display(tipo_dropdown, nome_dropdown, pollutant_dropdown, button, output)

Dropdown(description='Tipo:', options=('Comune', 'Provincia'), value='Comune')

Dropdown(description='Nome:', options=('Abbadia Cerreto', 'Abbadia Lariana', 'Abbiategrasso', 'Acquafredda', '…

Dropdown(description='Inquinante:', options=('Ammoniaca', 'Arsenico', 'Benzene', 'Benzo(a)pirene', 'Biossido d…

Button(description='Mostra dati', style=ButtonStyle())

Output()